In [21]:
from bs4 import BeautifulSoup, SoupStrainer, Tag
from gazpacho import Soup
import httplib2
import pandas as pd
import os
from random import randint
import requests
from selenium import webdriver

from functools import reduce
import time
from time import sleep

# Get url dataframe


In [25]:
url_dateframe = pd.read_csv("urls_20minutes.csv")

In [50]:
len(url_dateframe)

12381

In [26]:
if "Unnamed: 0" in url_dateframe.columns:
    url_dateframe.drop(columns="Unnamed: 0", inplace = True)

In [27]:
url_dateframe.drop_duplicates(inplace=True)
url_dateframe

,url_links
0,https://www.20minutes.fr/societe/2706043-20200...
1,https://www.20minutes.fr/monde/2706039-2020012...
2,https://www.20minutes.fr/societe/2709683-20200...
3,https://www.20minutes.fr/monde/2701675-2020012...
4,https://www.20minutes.fr/societe/2701619-20200...
5,https://www.20minutes.fr/monde/2701595-2020012...
6,https://www.20minutes.fr/monde/2703991-2020012...
7,https://www.20minutes.fr/lille/2702523-2020012...
8,https://www.20minutes.fr/sante/2704875-2020012...
9,https://www.20minutes.fr/monde/2704399-2020012...


In [30]:
article_dataframe = pd.DataFrame()
article_dataframe["Article_Date"] = ""
article_dataframe["Article_Title"] = ""
article_dataframe["Article_Summary"] = ""
article_dataframe["Article"] = ""
article_dataframe["Article_url"] = ""

In [31]:
article_dataframe

,Article_Date,Article_Title,Article_Summary,Article,Article_url


In [32]:
start = time.time()

In [48]:
df = url_dateframe["url_links"][11324:]
counter = len(url_dateframe["url_links"][11324:])

for url in df:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    soup_datetime = soup.find("p", {"class":"datetime"})
    date_time = soup_datetime.time.attrs["datetime"]
    article_time = pd.to_datetime(date_time).strftime("%Y-%m-%d")
    
    soup_title = soup.find("h1", {"class":"nodeheader-title"})
    article_title = soup_title.text

    soup_summary = soup.find("p", {"class":"hat"})
    article_summary = soup_summary.text.strip()
    
    soup_article = soup.find_all(["p","h2"], {"class":""})
    article_frame = []
    for paragraph in soup_article:
        
        text = paragraph.text.strip()
        article_frame.append(text)
        
    article = reduce(lambda para_left, para_right: para_left + " " + para_right, article_frame)
    article = article.strip()
    
    article_url = url
    
    article_dataframe = article_dataframe.append({
        "Article_Date": article_time, 
        "Article_Title": article_title,
        "Article_Summary": article_summary,
        "Article": article,
        "Article_url": article_url
    }, ignore_index=True)
    
    print(f"Article n* {counter} appended !")
    counter -= 1
    
    
print(time.time() - start)
article_dataframe.to_csv("Articles_20minutes.csv")

Article n* 1057 appended !
Article n* 1056 appended !
Article n* 1055 appended !
Article n* 1054 appended !
Article n* 1053 appended !
Article n* 1052 appended !
Article n* 1051 appended !
Article n* 1050 appended !
Article n* 1049 appended !
Article n* 1048 appended !
Article n* 1047 appended !
Article n* 1046 appended !
Article n* 1045 appended !
Article n* 1044 appended !
Article n* 1043 appended !
Article n* 1042 appended !
Article n* 1041 appended !
Article n* 1040 appended !
Article n* 1039 appended !
Article n* 1038 appended !
Article n* 1037 appended !
Article n* 1036 appended !
Article n* 1035 appended !
Article n* 1034 appended !
Article n* 1033 appended !
Article n* 1032 appended !
Article n* 1031 appended !
Article n* 1030 appended !
Article n* 1029 appended !
Article n* 1028 appended !
Article n* 1027 appended !
Article n* 1026 appended !
Article n* 1025 appended !
Article n* 1024 appended !
Article n* 1023 appended !
Article n* 1022 appended !
Article n* 1021 appended !
A

Article n* 744 appended !
Article n* 743 appended !
Article n* 742 appended !
Article n* 741 appended !
Article n* 740 appended !
Article n* 739 appended !
Article n* 738 appended !
Article n* 737 appended !
Article n* 736 appended !
Article n* 735 appended !
Article n* 734 appended !
Article n* 733 appended !
Article n* 732 appended !
Article n* 731 appended !
Article n* 730 appended !
Article n* 729 appended !
Article n* 728 appended !
Article n* 727 appended !
Article n* 726 appended !
Article n* 725 appended !
Article n* 724 appended !
Article n* 723 appended !
Article n* 722 appended !
Article n* 721 appended !
Article n* 720 appended !
Article n* 719 appended !
Article n* 718 appended !
Article n* 717 appended !
Article n* 716 appended !
Article n* 715 appended !
Article n* 714 appended !
Article n* 713 appended !
Article n* 712 appended !
Article n* 711 appended !
Article n* 710 appended !
Article n* 709 appended !
Article n* 708 appended !
Article n* 707 appended !
Article n* 7

Article n* 428 appended !
Article n* 427 appended !
Article n* 426 appended !
Article n* 425 appended !
Article n* 424 appended !
Article n* 423 appended !
Article n* 422 appended !
Article n* 421 appended !
Article n* 420 appended !
Article n* 419 appended !
Article n* 418 appended !
Article n* 417 appended !
Article n* 416 appended !
Article n* 415 appended !
Article n* 414 appended !
Article n* 413 appended !
Article n* 412 appended !
Article n* 411 appended !
Article n* 410 appended !
Article n* 409 appended !
Article n* 408 appended !
Article n* 407 appended !
Article n* 406 appended !
Article n* 405 appended !
Article n* 404 appended !
Article n* 403 appended !
Article n* 402 appended !
Article n* 401 appended !
Article n* 400 appended !
Article n* 399 appended !
Article n* 398 appended !
Article n* 397 appended !
Article n* 396 appended !
Article n* 395 appended !
Article n* 394 appended !
Article n* 393 appended !
Article n* 392 appended !
Article n* 391 appended !
Article n* 3

Article n* 112 appended !
Article n* 111 appended !
Article n* 110 appended !
Article n* 109 appended !
Article n* 108 appended !
Article n* 107 appended !
Article n* 106 appended !
Article n* 105 appended !
Article n* 104 appended !
Article n* 103 appended !
Article n* 102 appended !
Article n* 101 appended !
Article n* 100 appended !
Article n* 99 appended !
Article n* 98 appended !
Article n* 97 appended !
Article n* 96 appended !
Article n* 95 appended !
Article n* 94 appended !
Article n* 93 appended !
Article n* 92 appended !
Article n* 91 appended !
Article n* 90 appended !
Article n* 89 appended !
Article n* 88 appended !
Article n* 87 appended !
Article n* 86 appended !
Article n* 85 appended !
Article n* 84 appended !
Article n* 83 appended !
Article n* 82 appended !
Article n* 81 appended !
Article n* 80 appended !
Article n* 79 appended !
Article n* 78 appended !
Article n* 77 appended !
Article n* 76 appended !
Article n* 75 appended !
Article n* 74 appended !
Article n* 7

In [57]:
article_dataframe.drop_duplicates(inplace=True)
article_dataframe

,Article_Date,Article_Title,Article_Summary,Article,Article_url
0,2020-01-29,C'est l'heure du BIM : Nouvelle journée de grè...,ACTUALITES « 20 Minutes » compile pour vous le...,Vous avez manqué les infos de ce début de mati...,https://www.20minutes.fr/societe/2706043-20200...
1,2020-01-29,Coronavirus: Plus de 1.700 nouveaux cas en Chi...,LIVE Après la confirmation d’un quatrième cas ...,L’ESSENTIEL EN VIDEO POUR EN SAVOIR PLUS 3h00...,https://www.20minutes.fr/monde/2706039-2020012...
2,2020-02-03,C’est l’heure du BIM : Coronavirus plus meurtr...,ACTUALITES « 20 Minutes » compile pour vous le...,Vous avez manqué les infos de ce début de mati...,https://www.20minutes.fr/societe/2709683-20200...
3,2020-01-23,Coronavirus : La Chine confirme un 26e décès,EPIDEMIE Revivez avec nous les évènements auto...,L’ESSENTIEL POUR EN SAVOIR PLUS VIDEO 07h44 :...,https://www.20minutes.fr/monde/2701675-2020012...
4,2020-01-23,C'est l'heure du BIM : Le Coronavirus met Wuha...,ACTUALITES « 20 Minutes » compile pour vous le...,Vous avez manqué les infos de ce début de mati...,https://www.20minutes.fr/societe/2701619-20200...
5,2020-01-23,Coronavirus: Vols et trains suspendus au dépar...,"SANTE L'OMS, qui a prolongé d'une journée sa r...","Ce n’est pas une quarantaine, mais on s’en rap...",https://www.20minutes.fr/monde/2701595-2020012...
6,2020-01-26,Coronavirus : Les expatriés français de Wuhan ...,SANTÉ La ville chinoise de 11 millions d’habit...,Leur ville a été mise en quarantaine pour caus...,https://www.20minutes.fr/monde/2703991-2020012...
7,2020-01-26,Coronavirus : Quatre étudiantes de l’universit...,"SOCIAL Coronavirus oblige, quatre étudiantes e...","Elles sont rentrées, samedi, par avion. Quatre...",https://www.20minutes.fr/lille/2702523-2020012...
8,2020-01-27,"Coronavirus : Vol direct, quarantaine, surveil...","RISQUE S’ils le souhaitent, les Français insta...",Une épidémie de coronavirus qui progresse sur ...,https://www.20minutes.fr/sante/2704875-2020012...
9,2020-01-27,Coronavirus : Le Premier ministre chinois en v...,CHINE Il s’agit de la première visite dans la ...,Le Premier ministre chinois était en visite ce...,https://www.20minutes.fr/monde/2704399-2020012...


In [54]:
article_dataframe.to_csv("Articles_20minutes.csv", sep=(","))

# Get articles in json script format ? 
## idk if good idea

In [ ]:
urls_test = url_dateframe["url_links"][:100]

text_df = pd.DataFrame()
text_df["Article"] = ""
text_df

for url in urls_test:
    
    page = requests.get(test_url50)
    soup = BeautifulSoup(page.content, "html.parser")
    soup_summary = soup.find("script", {"type":"application/ld+json"})
    script = soup_summary.contents
    text_df = text_df.append({"Article": script}, ignore_index=True)

In [ ]:
text_df.head()